In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
df = pd.read_csv(r"C:\Users\檔名.csv")  # 或 pd.read_pickle('your_data.pkl')

In [ ]:
from geopy.distance import geodesic

# 使用官方提供的師大綜合大樓站牌座標
target_lat, target_lon = 25.026622, 121.53006  # 注意：緯度在前，經度在後

# 加入距離欄位（單位：公尺）
df['Distance'] = df.apply(
    lambda row: geodesic((row['PositionLat'], row['PositionLon']), (target_lat, target_lon)).meters
    if pd.notnull(row['PositionLat']) and pd.notnull(row['PositionLon']) else np.nan,
    axis=1
)

In [ ]:
# 把 e_time 轉成 datetime 格式
df['e_time'] = pd.to_datetime(df['e_time'])

# 拆出時間特徵
df['Hour'] = df['e_time'].dt.hour
df['Minute'] = df['e_time'].dt.minute
df['Weekday'] = df['e_time'].dt.weekday

In [ ]:
# 假設你的 DataFrame 名叫 df
df_filtered = df[df['Distance'] <= 20000].copy()

In [ ]:
# 先過濾 true_arrival_sec < 5000 的資料
df_limited = df_filtered[df_filtered['true_arrival_sec'] < 5000]

# 模型 A（含 EstimateTime）
feature_cols_A = ['Speed', 'rt_delay_sec', 'Hour', 'Minute', 'Weekday', 
                  'peak', 'daytype', 'rain', 'temp', 'wind', 'EstimateTime']
X_A = df_limited[feature_cols_A]
y_A = df_limited['true_arrival_sec']

# 模型 B（不含 EstimateTime）
feature_cols_B = ['Speed', 'rt_delay_sec', 'Distance', 'Hour', 'Minute', 'Weekday', 
                  'peak', 'daytype', 'rain', 'temp', 'wind']
X_B = df_limited[feature_cols_B]
y_B = df_limited['true_arrival_sec']

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def run_lr_model(X, y, feature_names, model_name):
    maes, rmses, r2s, models = [], [], [], []

    for i in range(20):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, shuffle=True, random_state=i
        )

        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        maes.append(mean_absolute_error(y_test, y_pred))
        rmses.append(np.sqrt(mean_squared_error(y_test, y_pred)))
        r2s.append(r2_score(y_test, y_pred))
        models.append(model)

    # 計算平均與找出最接近平均的那次
    avg_mae, avg_rmse, avg_r2 = np.mean(maes), np.mean(rmses), np.mean(r2s)
    distances = [(mae - avg_mae)**2 + (rmse - avg_rmse)**2 + (r2 - avg_r2)**2
                 for mae, rmse, r2 in zip(maes, rmses, r2s)]
    best_index = int(np.argmin(distances))
    best_model = models[best_index]

    # 印出結果
    print(f"\n📘 {model_name}：")
    print(f"平均 MAE: {avg_mae:.2f} ± {np.std(maes):.2f}")
    print(f"平均 RMSE: {avg_rmse:.2f} ± {np.std(rmses):.2f}")
    print(f"平均 R²: {avg_r2:.4f} ± {np.std(r2s):.4f}")
    print(f"\n🔍 最佳模型係數與截距（第 {best_index} 次）：")
    for name, coef in zip(feature_names, best_model.coef_):
        print(f"{name}: {coef:.2f}")
    print(f"截距（intercept）: {best_model.intercept_:.2f}")
    
# 執行模型 A
run_lr_model(X_A, y, feature_cols_A, "Linear Regression 模型 A（含 EstimateTime）")

# 執行模型 B
run_lr_model(X_B, y, feature_cols_B, "Linear Regression 模型 B（不含 EstimateTime）")


In [ ]:
# 模型 A 的係數與截距
print("\n📘 模型 A 係數：")
for name, coef in zip(feature_cols_A, lr_A.coef_):
    print(f"{name}: {coef:.2f}")
print(f"截距（intercept）: {lr_A.intercept_:.2f}")

# 模型 B 的係數與截距
print("\n📙 模型 B 係數：")
for name, coef in zip(feature_cols_B, lr_B.coef_):
    print(f"{name}: {coef:.2f}")
print(f"截距（intercept）: {lr_B.intercept_:.2f}")

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 加入常數項
X_with_const = sm.add_constant(X_A)

# 計算每個特徵的 VIF 值
vif = pd.DataFrame()
vif["feature"] = X_A.columns
vif["VIF"] = [variance_inflation_factor(X_with_const.values, i+1) for i in range(X_A.shape[1])]

print(vif.sort_values(by="VIF", ascending=False))

# 加入常數項
X_with_const = sm.add_constant(X_B)

# 計算每個特徵的 VIF 值
vif = pd.DataFrame()
vif["feature"] = X_B.columns
vif["VIF"] = [variance_inflation_factor(X_with_const.values, i+1) for i in range(X_B.shape[1])]

print(vif.sort_values(by="VIF", ascending=False))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 計算相關係數
corr_matrix = X_A.corr()

# 畫出 heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("特徵之間的相關性矩陣")
plt.show()

In [ ]:
# 模型 A
r2 = r2_score(y_test_A, y_pred_A)
n = X_test_A.shape[0]
p = X_test_A.shape[1]

r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f"模型 A Adjusted R²: {r2_adj:.4f}")

# 模型 B
r2 = r2_score(y_test_B, y_pred_B)
n = X_test_B.shape[0]
p = X_test_B.shape[1]

r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f"模型 B Adjusted R²: {r2_adj:.4f}")


In [ ]:
import joblib
joblib.dump(X_test_A, r'C:\Users\欲存成檔名.pkl')
joblib.dump(X_test_B, r'C:\Users\欲存成檔名.pkl')